In [9]:
#我的bbox
import math

def euclidean_distance(p1, p2):
    '''
    计算两个点的欧式距离
    '''
    y1,x1 = p1
    y2,x2 = p2
    return math.sqrt((x1-x2)**2 + (y1-y2)**2)

#我的bbox
import math
class BBox:
    '''
    业界规范：BBox[x,y,r,b] xy:左上角xy坐标  rb:右下角xy坐标
    '''
    def __init__(self, x, y, r, b, score=0):

        self.x, self.y, self.r, self.b, self.score = x, y,r,b ,score
        self.height = b - y +1
        self.width = r - x  +1

    def __repr__(self):
        return f"{self.x:.2f}, {self.y:.2f}, {self.r:.2f}, {self.b:.2f}, {self.score:.2f}"


    def __and__(self, other):
        '''
        计算box和other的交集
        '''
        x_max = min(self.r, other.r)
        y_max = min(self.b, other.b)
        x_min = max(self.x, other.x)
        y_min = max(self.y, other.y)
        
        cross_box  = BBox(x_min, y_min, x_max, y_max) #!!!
        if cross_box.width<= 0 or cross_box.height <= 0:
            return 0
        
        return cross_box.area

    def __or__(self, other):
        '''
        计算box和other的并集
        '''
        cross = self & other
        union = self.area + other.area - cross

        return union

    def __xor__(self, other):
        cross = self & other
        union = self | other
        return cross / (union + 1e-6)

    def boundof(self, other):
        '''
        计算box和other的边缘外包框，使得2个box都在框内的最小矩形
        '''
        x_min = min(self.x, other.x)
        y_min = min(self.y, other.y)
        x_max = max(self.r, other.r)
        y_max = max(self.b, other.b)

        return BBox(x_min, y_min, x_max, y_max)

    def center_dist(self,other):
        '''
        计算两个box的中心点距离
        '''
        return euclidean_distance(self.center,other.center)
    
    def bound_diagonal_dist(self,other):
        '''
        计算两个box的bound的对角线距离
        '''
        p1 = min(self.x,other.x),min(self.y,other.y)
        p2 = max(self.r,other.r),max(self.b,other.b)
        return euclidean_distance(p1,p2)

    @property
    def location(self):
        return self.x ,self.y, self.r, self.b
    
    @property
    def area(self):
        return self.height * self.width
    
    @property
    def center(self):
        return [(self.x + self.r)/2,(self.y+self.b)/2]


In [10]:
box = BBox(10, 10, 100, 200)
print(box)

10.00, 10.00, 100.00, 200.00, 0.00


In [11]:
def IoU(box1: BBox, box2: BBox):
    return box1 ^ box2

实现GIoU函数，计算giou = GIoU(box1, box2)，并打印giou
- ![title](https://www.zhihu.com/equation?tex=GIoU+%3D+IoU+-+%5Cfrac%7B%5Cleft%7C++A_%7Bc%7D+-+U+%5Cright%7C%7D%7B%5Cleft%7C+A_%7Bc%7D+%5Cright%7C%7D)

In [12]:
def GIoU(box1: BBox, box2: BBox):

    Ac = box1.boundof(box2).area
    union = box1 | box2
    IoU_value = IoU(box1, box2)

    return IoU_value - abs(Ac - union) / abs(Ac)

实现DIoU函数，计算diou = DIoU(box1, box2)，并打印diou
 - ![title](https://www.zhihu.com/equation?tex=L_%7BDIoU%7D+%3D+1-+IoU+%2B%5Cfrac%7B%5Crho%5E%7B2%7D%5Cleft%28+b%2Cb%5E%7Bgt%7D+%5Cright%29%7D%7Bc%5E%7B2%7D%7D+)

In [13]:
def DIoU(box1: BBox, box2: BBox):
    d = box1.center_dist(box2)
    c = box1.bound_diagonal_dist(box2)
    return IoU(box1, box2) - d**2 / c**2

实现CIoU函数，计算ciou = CIoU(box1, box2)，并打印ciou
 - ![title](https://www.zhihu.com/equation?tex=L_%7BCIoU%7D+%3D+1-+IoU+%2B%5Cfrac%7B%5Crho%5E%7B2%7D%5Cleft%28+b%2Cb%5E%7Bgt%7D+%5Cright%29%7D%7Bc%5E%7B2%7D%7D+++%2B+%5Calpha%5Cupsilon)  
 - ![title](https://www.zhihu.com/equation?tex=%5Calpha%3D++%5Cfrac%7B%5Cupsilon%7D%7B%5Cleft%28+1-IoU+%5Cright%29%2B%5Cupsilon%7D+)   
 - ![title](https://www.zhihu.com/equation?tex=%5Cupsilon%3D%5Cfrac%7B4%7D%7B%5Cpi%5E%7B2%7D%7D%5Cleft%28+arctan%5Cfrac%7Bw%5E%7Bgt%7D%7D%7Bh%5E%7Bgt%7D%7D+-+arctan%5Cfrac%7Bw%7D%7Bh%7D+%5Cright%29%5E%7B2%7D)

In [14]:
def CIoU(box1: BBox, box2: BBox):

    v = 4 / math.pi**2 * (math.atan(box1.width / box1.height) -
                          math.atan(box2.width / box2.height))**2
    alpha = v / (1 - IoU(box1, box2) + v)
    return DIoU(box1, box2) - alpha * v

In [15]:
box1 = BBox(*[10, 10, 100, 200])
box2 = BBox(*[50, 50, 150, 180])
# box1 = BBox(*[0,0,2,2])
# box2 = BBox(*[1,1,3,3])


print(GIoU(box1,box2),
DIoU(box1,box2),
CIoU(box1,box2))



0.16778184645181238 0.24102682779151774 0.24057637895114647
